In [ ]:
# selenium 예제 15: # 등업을 해야한다  #게시판이름 메모장에 복사하기 
# 네이버에 로그인 후
# 네이버 카페 주소 입력받고
# 게시판 이름 입력 받고
#원하는 건수를 입력받아서
#각 페이지에 있는 블로그의 상세 텍스트를 모두 수집한 후
#현재 날짜와 시간으로 폴더를 만들고
#그 폴더아래 현재 날짜와 시간으로 파일명을 만들어 저장하기

# 크롬일 경우 네이버 로그인 한 후 브라우저 하단의 플래시 경고창을 반드시 닫고 시작해야 합니다! 

#참고 사이트 : http://yahwang.tk/posts/4

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
import time
import sys
import re
import math
import numpy  # pip install numpy 를 해야 사용가능함
import pandas as pd  # pip install pandas 해야 사용가능하고 numpy 가 먼저 설치되어 있어야
import xlwt # excel 형태로 저장하기 위해 필요함. pip install xlwt 해야 사용가능함
import time
import os
import random

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 연습문제 : 네이버 카페의 게시물과 댓글 정보 수집하기")
print("=" *80)

# Step 3. 네이버에 로그인 합니다

s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://www.naver.com")

time.sleep(20)  ## 사용자가 로그인 할 때까지 20초 기다립니다

cafe_name = input('1.크롤링할 카페 이름을 입력하세요: ')
cafe_url = 'http://cafe.naver.com/'+ cafe_name
    
gesipan_name = input('2.조회할 게시판의 이름을 입력하세요(기본값:전체글보기): ')
if gesipan_name == '':
        gesipan_name = '전체글보기'
    
print("=" *80)
choice = int(input('''
[조회 방법 선택하기]
    
1.특정 키워드와 조회기간 입력하여 조회하기
2.오늘을 기준으로 전체 건수만 입력하여 조회하기 
3.특정 게시물을 지정하여 내용과 댓글 정보 수집하기
    
위 작업 중 원하는 작업을 선택하세요:  '''))

print("=" *80)
print("\n")

    
if choice == 1 :
        start_date=input('1.조회 시작 날짜를 입력하세요(기본값:2018-01-01): ')
        if start_date == '' :
            start_date = '2018-01-01'
            
        end_date=input('2.조회 종료 날짜를 입력하세요(기본값:2018-12-31): ')
        if end_date == '' :
            end_date = '2018-12-31'

        query_txt=''
        while ( query_txt==''):
            query_txt = input('3.검색할 키워드를 반드시 입력하세요: ')
    
        try :
            cnt = int(input('4.크롤링 할 건수는 몇건입니까?(기본건수:15건): '))
        except ValueError :
            cnt = 15

        page_cnt = math.ceil(cnt / 15)  # 주어진 검색 건수를 크롤링 하기 위한 전체 페이지 번호
    
        f_dir = "c:\\temp\\"
        #f_dir=input('4.파일이 저장될 경로만 쓰세요(예: c:\\temp\\ ) : ')
        if f_dir == '' :
            f_dir = "c:\\temp\\"
        

        print("=" *80)
        print("입력하신 정보로 총 %s 페이지의 데이터 수집을 시작합니다================================" %page_cnt)
        print("\n")
  
        driver.get(cafe_url)    

        # 페이지 번호 만들기 - 10 페이지가 지나면 11 페이지가 없고 다음 버튼이 나오기 때문에
        # 10 페이지 후에 11 페이지 대신 다음 페이지 누르려고 만든 변수

        page_no=[]

        for i in range(10,cnt) :
            if i % 10 == 0 :
                #page_no.append(i+1)
                page_no.append(i+1)
        #print(page_no)
        

        # Step 3. 카페 페이지를 엽니다
        driver.get(cafe_url)
        time.sleep(1)

        now = time.localtime()
        s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
        os.chdir(f_dir)
        os.makedirs(f_dir+s+'-'+cafe_name)
        os.chdir(f_dir+s+'-'+cafe_name)
        f_name=s+'-'+cafe_name+'.txt'
        ff_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.txt'
        fx_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.xls'
        fc_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.csv'

        def scroll_down(driver):
          driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
          time.sleep(1)


        i = 1
        while (i <= 2):
              scroll_down(driver) 
              i += 1
        time.sleep(1)

        # 플래시 경고창을 닫기
        #driver.find_element_by_xpath("""/html/body/div[2]/span[3]/button/img""").click( )

        time.sleep(2)

        #선택한 게시판으로 이동하기
        driver.find_element_by_link_text("%s" %gesipan_name).click()   #여행후기
        #driver.find_element_by_xpath("""//*[@id="menuLink0"]""").click( )   # 전체글보기 메뉴 선택
        time.sleep(2) 

        

        #Step 7. 검색 조건과 기간 버튼 클릭
        driver.switch_to.frame('cafe_main')
        #driver.switch_to.frame('wordchk')

        driver.find_element_by_id("currentSearchDate").click( )
        time.sleep(1)

        # 직접 입력 시작 날짜 클릭
        s_date = driver.find_element_by_xpath("""//*[@id="input_1"]""")     
        driver.find_element_by_xpath("""//*[@id="input_1"]""").click()
        s_date.clear()
        time.sleep(1)

        for c in start_date:
            s_date.send_keys(c)
            time.sleep(0.3)

        e_date = driver.find_element_by_xpath("""//*[@id="input_2"]""")
        driver.find_element_by_xpath("""//*[@id="input_2"]""").click()
        e_date.clear()
        time.sleep(1)

        for c in end_date:
            e_date.send_keys(c)
            time.sleep(0.3)

        driver.find_element_by_id("""btn_set""").click()

        time.sleep(2)
        driver.find_element_by_name("query").send_keys(query_txt)

        # 검색버튼 클릭
        driver.find_element_by_xpath("""//*[@id="main-area"]/div[7]/form/div[3]/button""").click() # 여행후기 게시판 검색버튼
        # driver.find_element_by_xpath("""//*[@id="main-area"]/div[7]/form/div[3]/button""").click() #전체글보기 검색버튼
        time.sleep(2)    

   

        # 검색 결과 페이지의 상세 내역을 추출합니다

        #원본 게시물의 URL 주소를 저장할 변수
        article_urls_2=[]

        click_count= 1       # 페이지 번호 넘길 때 사용할 변수
        current_page = 1
        count = 1

        item_count = 0   # 총 수집한 게시물의 건수 저장 변수

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 현재 페이지 - 1 페이지의 게시물의 원본 URL 목록 만들기
        try : 
            all_list = soup.find('div','article-board result-board m-tcol-c')
        except :
            print("검색된 게시물이 없습니다")
        else :
            a_list = all_list.find('table').find('tbody').find_all('tr')
        

        for i in a_list :
            try :
                a =i.find('div','inner_list').find('a')
            except :
                #print("현재 페이지에서 에러가 발생했습니다")
                continue
            else :
                a_url = a['href']
                #print(a_url)
                article_urls_2.append(a_url)
                item_count += 1
                      
            if item_count >= cnt :
                break
                
        print("현재 %s 페이지에서 %s 건의 정보를 추출했습니다^^" %(click_count,len(article_urls_2)))
   
        click_count += 1     # 페이지 번호 1 증가
              
        # 2 페이지로 넘어가야 할 경우 아래 코드 실행
        if cnt > 15:
        
            driver.find_element_by_link_text("""%s""" %click_count).click()  # 2페이지 클릭
        
            while ( click_count <= page_cnt ) :
                
                time.sleep(1)
                
                #driver.switch_to.frame('cafe_main')
                
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                  
                b = soup.find('div','article-board result-board m-tcol-c')
                b_list = b.find('table').find('tbody').find_all('tr')                      
               

                for j in b_list :
                    try :
                        a =j.find('div','inner_list').find('a')                        
                    except AttributeError:
                        #print("현재 페이지에서 에러가 발생했습니다")
                        continue
                    else :    
                        a_url = a['href']
                        #print(a_url)
                        article_urls_2.append(a_url)
                        item_count += 1
                      
                    if item_count >= cnt :
                        break
                    
                print("현재 %s 페이지까지 총 %s 건의 URL 정보를 수집했습니다~~~^^" %(click_count,len(article_urls_2)))

             # 11 페이지 번호가 화면에 없기 때문에 > 기호를 누르게 코드를 작성합니다
                i = math.floor(current_page/10)
                click_count += 1 
            
                try :
                    if click_count == 1000 :
                         click_count2 = "{:,}".format(click_count)
                         driver.find_element_by_link_text("""%s""" %click_count2).click()
                         break
                    elif click_count == page_no[i] :                      
                        driver.find_element_by_link_text("""다음""").click()
                    elif click_count > 1000 :
                        print("최대 1000 페이지까지만 수집할 수 있습니다")
                        break
                    else :
                        driver.find_element_by_link_text("""%s""" %click_count).click()
                   
                    current_page += 1
                except :
                    print("총 %s 페이지까지 URL 정보 수집을 완료했습니다" %click_count)
                    break
            
                time.sleep(random.randrange(1,30))
                            
           ## 실제 게시글의 내용 추출하기

        if cnt > len(article_urls_2) :
            cnt = len(article_urls_2)

        print("전체 검색 결과 건수 :",len(article_urls_2),"건")
        print("실제 최종 출력 건수",cnt)

        print("\n")
        print("크롤링 할 총 페이지 번호: ", click_count-1)
        print("=" *80)
        print("\n")

        print("총 %s 건의 데이터를 추출하겠습니다~~~~~~~" %len(article_urls_2))

        no = 1        # 데이터 건수 세기 
    
        no2=[]           # 목록 번호 저장 변수
        write_no2=[]     # 게시글번호 저장 변수
        url_2=[]         # 원본 URL 저장 변수
        writer_id2=[]    # 작성자 이름 저장 변수
        title2=[]        # 제목 저장 변수
        wdate2=[]        # 작성 일자 저장 변수
        content2=[]      # 내용 저장 변수
        reple_cnt2=[]   # 리플 갯수 저장 변수
        view_cnt2=[]

        # 결과를 화면에 출력하기

        print("\n")
        print("게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" )
        print("\n")
        count = 1
    
        no=[]           # 목록 번호 저장 변수
        write_no=[]     # 게시글 번호 저장 변수
        writer_id=[]    # 작성자 이름 저장 변수
        title=[]        # 제목 저장 변수
        wdate=[]        # 작성 일자 저장 변수
        content=[]      # 내용 저장 변수
        reple_cnt=[]    # 리플 갯수 저장 변수
        view_cnt=[]     # 조회수 저장변수

        for article in article_urls_2:
       
            full_url = cafe_url+'/'+article
            driver.get(full_url)
        
            no2.append(no)            # 1.글번호 컬럼 내용 추가
            url_2.append(full_url)    #2. 원본 URL 주소 컬럼 내용 추가
        
            try :
                driver.switch_to.frame('cafe_main')
            except  :
                continue
            else :    
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
     
                f = open(ff_name, 'a',encoding='UTF-8')
            
                all_content = soup.find('div','inbox')
                #print(all_content)
            
                print("%s번째 게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" %count )
            
                #1.게시글번호 추출 - write_no 변수
                w_no_1 = all_content.find('div','etc-box').find('div','fr')
                w_no_2 = w_no_1.select('table > tbody > tr > td > span > a')
                w_no_3 = w_no_2[0]['href'].split('/')
                write_no = w_no_3[4]
                write_no2.append(write_no)
            
                f.write('------------------------------------------------------'+"\n")
                f.write("%s번째 게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" %count + "\n" )
                f.write('1.게시글번호:'+ write_no + "\n")
            

               #2.게시글에서 작성자 추출 - writer_id 변수
                w_name_1 = all_content.find('div','etc-box').find('div','fl')
                w_name_2 = w_name_1.select('table > tbody > tr > td > table > tbody > tr > td ')
                writer_id = w_name_2[1].get_text()
                #print(writer_name)
                writer_id2.append(writer_id)
                f.write('2.작성자닉네임:'+ writer_id + "\n")
            

                #3.게시글에서 제목 추출 - title 변수
            
                title_1 = all_content.find('div','tit-box').find('div','fl')
                title_2 = title_1.select('table > tbody > tr > td > span ')
                title = title_2[0].get_text().replace("\n","").strip()
                #print(title)
                title2.append(title)
                f.write('3.게시글제목:'+ title + "\n")
                     

                #4. 게시글에서 작성일자 추출 - wdate 변수
                wdate_1 = all_content.find('div','tit-box').find('div','fr')
                wdate_2 = wdate_1.select('table > tbody > tr > td ')
                wdate = wdate_2[1].get_text().replace("\n","").strip()
                #print(wdate)
                wdate2.append(wdate)
                f.write('4.작성일자:'+ wdate + "\n")
            

                #5. 본문 내용을 출력합니다 - content 변수 사용
    
                content_1 = all_content.select('#tbody')
                content_2 = content_1[0].get_text().replace("\n","").strip()
                print(content_2)
                content2.append(content_2)
                f.write('5.본문내용:'+ content_2 + "\n")

              # 6.게시글에서 댓글수 추출 - reple_cnt 변수사용
    
                all_reple = soup.select('#cmtMenu')
                reple_1 = all_reple[0].select('div > table > tbody > tr > td > a')
                reple_2 = reple_1[0].get_text()
                reple_3 = reple_2.replace(",","")
                reple_4 = re.search("\d+",reple_3)
                try :
                    reple_cnt = int(reple_4.group())
                except :
                    reple_cnt = 0
                #print(reple_cnt)   
                reple_cnt2.append(reple_cnt)
                f.write('6.댓글수:'+ str(reple_cnt) + "\n")

                # 7.게시글에서 조회수  추출 - view_cnt 변수사용
    
                view_1 = all_reple[0].select('div > table > tbody > tr')
                view_2 = view_1[0].find_all('td')
                view_3 = view_2[4].get_text()
                view_4 = view_3.replace(",","")
                view_5 = re.search("\d+",view_4)
                try :
                    view_cnt = int(view_5.group())
                except :
                    view_cnt = 0
                #print(view_cnt)   
                view_cnt2.append(view_cnt)
                f.write('7.조회수:'+ str(view_cnt) + "\n")
                time.sleep(random.randrange(1,30)) 
                f.write("\n")
            
                print("\n")

                count += 1

elif choice == 2 :      # 현재 날짜 기준으로 건수를 입력받아서 수집하는 부분
        try :
            cnt = int(input('1.크롤링 할 건수는 몇건입니까?(기본값:15): '))
        except ValueError :
            cnt = 15
        #cnt = 2500
        page_cnt = math.ceil(cnt / 15)  # 주어진 검색 건수를 크롤링 하기 위한 전체 페이지 번호
    
        f_dir = "c:\\temp\\"
        #f_dir=input('4.파일이 저장될 경로만 쓰세요(예: c:\\temp\\ ) : ')
        if f_dir == '' :
            f_dir = "c:\\temp\\"

        print("=" *80)
        print("입력하신 정보로 총 %s 페이지의 데이터 수집을 시작합니다============================" %page_cnt)
        print("\n")

        # 페이지 번호 만들기 - 10 페이지가 지나면 11 페이지가 없고 다음 버튼이 나오기 때문에
        # 10 페이지 후에 11 페이지 대신 다음 페이지 누르려고 만든 변수

        page_no=[]

        for i in range(10,cnt) :
            if i % 10 == 0 :
                #page_no.append(i+1)
                page_no.append(i+1)

        # Step 3. 카페 페이지를 엽니다
        driver.get(cafe_url)
        time.sleep(1)

        now = time.localtime()
        s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
        os.chdir(f_dir)
        os.makedirs(f_dir+s+'-'+cafe_name)
        os.chdir(f_dir+s+'-'+cafe_name)
        f_name=s+'-'+cafe_name+'.txt'
        ff_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.txt'
        fx_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.xls'
        fc_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.csv'

        def scroll_down(driver):
          driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
          time.sleep(1)


        i = 1
        while (i <= 2):
              scroll_down(driver) 
              i += 1
        time.sleep(1)

        # 플래시 경고창을 닫기
        #driver.find_element_by_xpath("""/html/body/div[2]/span[3]/button/img""").click( )

        time.sleep(2)

        #선택한 게시판으로 이동하기
        driver.find_element_by_link_text("%s" %gesipan_name).click()   # 여행후기
        #driver.find_element_by_xpath("""//*[@id="menuLink0"]""").click( )   # 전체글보기 메뉴 선택
      
        time.sleep(2) 
       

        # 검색 결과 페이지에서 원본 페이지의 URL 주소를 먼저 추출합니다        
        driver.switch_to.frame('cafe_main')
        
        #원본 게시물의 URL 주소를 저장할 변수
        article_urls_2=[]

        click_count= 1       # 페이지 번호 넘길 때 사용할 변수
        current_page = 1
        count = 1

        item_count = 0   # 총 수집한 게시물의 건수 저장 변수

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # 현재 페이지 - 1 페이지의 게시물의 원본 URL 목록 만들기
        all_list = soup.find_all('div','article-board m-tcol-c')
        a_list = all_list[1].find('table').find('tbody').find_all('tr')
        

        for i in a_list :
            try :
                a =i.find('div','inner_list').find('a')
            except :
                #print("현재 페이지에서 에러가 발생했습니다")
                continue
            else :
                a_url = a['href']
                #print(a_url)
                article_urls_2.append(a_url)
                item_count += 1
                      
            if item_count >= cnt :
                break
                
        print("현재 %s 페이지에서 %s 건의 정보를 추출했습니다^^" %(click_count,len(article_urls_2)))
   
        click_count += 1     # 페이지 번호 1 증가
              
        # 2 페이지로 넘어가야 할 경우 아래 코드 실행
        if cnt > 15:
        
            driver.find_element_by_link_text("""%s""" %click_count).click()  # 2페이지 클릭
        
            while ( click_count <= page_cnt ) :
                
                time.sleep(1)
                
                #driver.switch_to.frame('cafe_main')
                
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                  
                b = soup.find_all('div','article-board m-tcol-c')
                b_list = b[1].find('table').find('tbody').find_all('tr')                       
               

                for j in b_list :
                    try :
                        a =j.find('div','inner_list').find('a')                        
                    except AttributeError:
                        #print("현재 페이지에서 에러가 발생했습니다")
                        continue
                    else :    
                        a_url = a['href']
                        #print(a_url)
                        article_urls_2.append(a_url)
                        item_count += 1
                      
                    if item_count >= cnt :
                        break
                    
                print("현재 %s 페이지까지 총 %s 건의 URL 정보를 수집했습니다~~~^^" %(click_count,len(article_urls_2)))

             # 11 페이지 번호가 화면에 없기 때문에 > 기호를 누르게 코드를 작성합니다
                i = math.floor(current_page/10)
                click_count += 1 
            
                try :
                    if click_count == 1000 :
                         click_count2 = "{:,}".format(click_count)
                         driver.find_element_by_link_text("""%s""" %click_count2).click()
                         break
                    elif click_count == page_no[i] :                      
                        driver.find_element_by_link_text("""다음""").click()
                    elif click_count > 1000 :
                        print("최대 1000 페이지까지만 수집할 수 있습니다")
                        break
                    else :
                        driver.find_element_by_link_text("""%s""" %click_count).click()
                   
                    current_page += 1
                except :
                    print("총 %s 페이지까지 URL 정보 수집을 완료했습니다" %click_count)
                    break
            
                time.sleep(random.randrange(1,30))
                            

 

       ## 실제 게시글의 내용 추출하기

        if cnt > len(article_urls_2) :
            cnt = len(article_urls_2)

        print("전체 검색 결과 건수 :",len(article_urls_2),"건")
        print("실제 최종 출력 건수",cnt)

        print("\n")
        print("크롤링 할 총 페이지 번호: ", click_count-1)
        print("=" *80)
        print("\n")

        print("총 %s 건의 데이터를 추출하겠습니다~~~~~~~" %len(article_urls_2))

        no = 1        # 데이터 건수 세기 
    
        no2=[]           # 목록 번호 저장 변수
        write_no2=[]     # 게시글번호 저장 변수
        url_2=[]         # 원본 URL 저장 변수
        writer_id2=[]    # 작성자 이름 저장 변수
        title2=[]        # 제목 저장 변수
        wdate2=[]        # 작성 일자 저장 변수
        content2=[]      # 내용 저장 변수
        reple_cnt2=[]   # 리플 갯수 저장 변수
        view_cnt2=[]

        # 결과를 화면에 출력하기

        print("\n")
        print("게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" )
        print("\n")
        count = 1
    
        no=[]           # 목록 번호 저장 변수
        write_no=[]     # 게시글 번호 저장 변수
        writer_id=[]    # 작성자 이름 저장 변수
        title=[]        # 제목 저장 변수
        wdate=[]        # 작성 일자 저장 변수
        content=[]      # 내용 저장 변수
        reple_cnt=[]    # 리플 갯수 저장 변수
        view_cnt=[]     # 조회수 저장변수

        for article in article_urls_2:
       
            full_url = cafe_url+'/'+article
            driver.get(full_url)
        
            no2.append(no)            # 1.글번호 컬럼 내용 추가
            url_2.append(full_url)    #2. 원본 URL 주소 컬럼 내용 추가
        
            try :
                driver.switch_to.frame('cafe_main')
            except  :
                continue
            else :    
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
     
                f = open(ff_name, 'a',encoding='UTF-8')
            
                all_content = soup.find('div','inbox')
                #print(all_content)
            
                print("%s번째 게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" %count )
            
                #1.게시글번호 추출 - write_no 변수
                w_no_1 = all_content.find('div','etc-box').find('div','fr')
                w_no_2 = w_no_1.select('table > tbody > tr > td > span > a')
                w_no_3 = w_no_2[0]['href'].split('/')
                write_no = w_no_3[4]
                write_no2.append(write_no)
            
                f.write('------------------------------------------------------'+"\n")
                f.write("%s번째 게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" %count + "\n" )
                f.write('1.게시글번호:'+ write_no + "\n")
            

               #2.게시글에서 작성자 추출 - writer_id 변수
                w_name_1 = all_content.find('div','etc-box').find('div','fl')
                w_name_2 = w_name_1.select('table > tbody > tr > td > table > tbody > tr > td ')
                writer_id = w_name_2[1].get_text()
                #print(writer_name)
                writer_id2.append(writer_id)
                f.write('2.작성자닉네임:'+ writer_id + "\n")
            

                #3.게시글에서 제목 추출 - title 변수
            
                title_1 = all_content.find('div','tit-box').find('div','fl')
                title_2 = title_1.select('table > tbody > tr > td > span ')
                title = title_2[0].get_text().replace("\n","").strip()
                #print(title)
                title2.append(title)
                f.write('3.게시글제목:'+ title + "\n")
                     

                #4. 게시글에서 작성일자 추출 - wdate 변수
                wdate_1 = all_content.find('div','tit-box').find('div','fr')
                wdate_2 = wdate_1.select('table > tbody > tr > td ')
                wdate = wdate_2[1].get_text().replace("\n","").strip()
                #print(wdate)
                wdate2.append(wdate)
                f.write('4.작성일자:'+ wdate + "\n")
            

                #5. 본문 내용을 출력합니다 - content 변수 사용
    
                content_1 = all_content.select('#tbody')
                content_2 = content_1[0].get_text().replace("\n","").strip()
                print(content_2)
                content2.append(content_2)
                f.write('5.본문내용:'+ content_2 + "\n")

              # 6.게시글에서 댓글수 추출 - reple_cnt 변수사용
    
                all_reple = soup.select('#cmtMenu')
                reple_1 = all_reple[0].select('div > table > tbody > tr > td > a')
                reple_2 = reple_1[0].get_text()
                reple_3 = reple_2.replace(",","")
                reple_4 = re.search("\d+",reple_3)
                try :
                    reple_cnt = int(reple_4.group())
                except :
                    reple_cnt = 0
                #print(reple_cnt)   
                reple_cnt2.append(reple_cnt)
                f.write('6.댓글수:'+ str(reple_cnt) + "\n")

                # 7.게시글에서 조회수  추출 - view_cnt 변수사용
    
                view_1 = all_reple[0].select('div > table > tbody > tr')
                view_2 = view_1[0].find_all('td')
                view_3 = view_2[4].get_text()
                view_4 = view_3.replace(",","")
                view_5 = re.search("\d+",view_4)
                try :
                    view_cnt = int(view_5.group())
                except :
                    view_cnt = 0
                #print(view_cnt)   
                view_cnt2.append(view_cnt)
                f.write('7.조회수:'+ str(view_cnt) + "\n")
                time.sleep(random.randrange(1,30)) 
                f.write("\n")
            
                print("\n")

                count += 1
                
elif choice == 3 :
    
        now = time.localtime()
        s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
        os.makedirs(f_dir+s+'-'+cafe_name)

        ff_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.txt'
        fx_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.xls'
        fc_name=f_dir+s+'-'+cafe_name+'\\'+s+'-'+cafe_name+'.csv'
    
        g_name = input('조회할 게시글의 주소를 입력하세요:')
        driver.get(g_name)
        time.sleep(2)
    
        driver.switch_to.frame('cafe_main')
   
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
     
        f = open(ff_name, 'a',encoding='UTF-8')
    
        all_content = soup.find('div','inbox')
        #print(all_content)
            
        print("요청하신 게시물의 내용과 댓글 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" )
        print("\n")
            
        #1.게시글번호 추출 - write_no 변수
        w_no_1 = all_content.find('div','etc-box').find('div','fr')
        w_no_2 = w_no_1.select('table > tbody > tr > td > span > a')
        w_no_3 = w_no_2[0]['href'].split('/')
        write_no = w_no_3[4]
        write_no2.append(write_no)
        print('1.게시글번호:',write_no)
            
        f.write('------------------------------------------------------'+"\n")
        f.write("%s번째 게시물의 수집을 시작합니다~~~~~~~~~~~~~~~~~~~~~~~~~~~~~" %count + "\n" )
        f.write('1.게시글번호:'+ write_no + "\n")
            

        #2.게시글에서 작성자 추출 - writer_id 변수
        w_name_1 = all_content.find('div','etc-box').find('div','fl')
        w_name_2 = w_name_1.select('table > tbody > tr > td > table > tbody > tr > td ')
        writer_id = w_name_2[1].get_text()
        print('2.작성자닉네임:',writer_id)
        writer_id2.append(writer_id)
        f.write('2.작성자닉네임:'+ writer_id + "\n")
            

        #3.게시글에서 제목 추출 - title 변수
            
        title_1 = all_content.find('div','tit-box').find('div','fl')
        title_2 = title_1.select('table > tbody > tr > td > span ')
        title = title_2[0].get_text().replace("\n","").strip()
        print('3.게시글제목:',title)
        title2.append(title)
        f.write('3.게시글제목:'+ title + "\n")
                     

        #4. 게시글에서 작성일자 추출 - wdate 변수
        wdate_1 = all_content.find('div','tit-box').find('div','fr')
        wdate_2 = wdate_1.select('table > tbody > tr > td ')
        wdate = wdate_2[1].get_text().replace("\n","").strip()
        print('4.작성일자:',wdate)
        wdate2.append(wdate)
        f.write('4.작성일자:'+ wdate + "\n")
            

        #5. 본문 내용을 출력합니다 - content 변수 사용
    
        content_1 = all_content.select('#tbody')
        content_2 = content_1[0].get_text().replace("\n","").strip()
        print('5.본문내용:',content_2)
        content2.append(content_2)
        f.write('5.본문내용:'+ content_2 + "\n")

        # 6.게시글에서 댓글수 추출 - reple_cnt 변수사용
    
        all_reple = soup.select('#cmtMenu')
        reple_1 = all_reple[0].select('div > table > tbody > tr > td > a')
        reple_2 = reple_1[0].get_text()
        reple_3 = reple_2.replace(",","")
        reple_4 = re.search("\d+",reple_3)
        try :
            reple_cnt = int(reple_4.group())
        except :
            reple_cnt = 0
        print('6.댓글수:',reple_cnt)   
        reple_cnt2.append(reple_cnt)
        f.write('6.댓글수:'+ str(reple_cnt) + "\n")

        # 7.게시글에서 조회수  추출 - view_cnt 변수사용
    
        view_1 = all_reple[0].select('div > table > tbody > tr')
        view_2 = view_1[0].find_all('td')
        view_3 = view_2[4].get_text()
        view_4 = view_3.replace(",","")
        view_5 = re.search("\d+",view_4)
        try :
            view_cnt = int(view_5.group())
        except :
            view_cnt = 0
        print('7.조회수:',view_cnt)   
        view_cnt2.append(view_cnt)
        f.write('7.조회수:'+ str(view_cnt) + "\n")
    
        f.write("\n")
    
        
        # 댓글  정보 추출
        review2_id=[]
        review2_content=[]
        review2_date=[]
        
        all_review= soup.select('#cmt_list > li')
        
        for b in range(0,len(all_review)) :
            try :
                review_1 = all_review[b].find('div','comm_cont').find('a','m-tcol-c _rosRestrict _nickUI').get_text().replace("작성자","").strip()
            except :
                continue
                
            b += 1
            print('-댓글 작성자:', review_1)
            review2_id.append(review_1)
            
            b -= 1
            try:
                review_2 = all_review[b].find('div','comm_cont').find('p','comm m-tcol-c').get_text().strip()
            except :
                continue
                
            b += 1
            print('-댓글 내용:', review_2)
            review2_content.append(review_2)
            
            b -= 1
            try :
                review_3 = all_review[b].find('div','comm_cont').find('span','date m-tcol-c filter-50').get_text().strip()
            except :
                continue
                
            b += 1
            print('-댓글 작성 날짜:' , review_3)
            review2_date.append(review_2)

# 결과 데이터프레임화

if choice == 1 or choice ==2 :
    cafe_df = pd.DataFrame()
    cafe_df['게시글번호']=write_no2
    cafe_df['게시글 URL']= pd.Series(url_2)
    cafe_df['작성자ID']=pd.Series(writer_id2)
    cafe_df['게시글제목']=pd.Series(title2)
    cafe_df['작성일자']=pd.Series(wdate2)     
    cafe_df['본문내용']=pd.Series(content2)
    cafe_df['댓글수']=pd.Series(reple_cnt2)
    cafe_df['조회수']=pd.Series(view_cnt2)   
elif choice == 3 :
    cafe_df = pd.DataFrame()
    cafe_df['게시글번호']=write_no2
    cafe_df['게시글 URL']= pd.Series(url_2)
    cafe_df['작성자ID']=pd.Series(writer_id2)
    cafe_df['게시글제목']=pd.Series(title2)
    cafe_df['작성일자']=pd.Series(wdate2)     
    cafe_df['본문내용']=pd.Series(content2)
    cafe_df['댓글수']=pd.Series(reple_cnt2)
    cafe_df['조회수']=pd.Series(view_cnt2)
    cafe_df['댓글작성자'] = pd.Series(review2_id)
    cafe_df['댓글내용']= pd.Series(review2_content)
    cafe_df['댓글작성일자']=pd.Series(review2_date)
    

# csv파일로 저장
cafe_df.to_csv(fc_name, encoding="utf-8-sig", index=False)

#엑셀 파일로 저장
cafe_df.to_excel(fx_name)

count -= 1
e_time = time.time( )
t_time = e_time - s_time

print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("txt 파일 저장 완료: 파일명 : %s " %ff_name)
print("csv 파일 저장 완료: 파일명 : %s " %fc_name)
print("xls 파일 저장 완료: 파일명 : %s " %fx_name)
print("=" *80)

# 요약 결과를 txt파일에 저장하기
orig_stdout = sys.stdout
f = open(f_name, 'a',encoding='UTF-8')
sys.stdout = f
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("=" *80)
sys.stdout = orig_stdout
f.close()

driver.close( )
    

 연습문제 : 네이버 카페의 게시물과 댓글 정보 수집하기
